# Knowledge Graph and Cypher Query

### CSV method
LOAD CSV WITH HEADERS FROM 'file:///stock2.csv' AS row   
MERGE (s:Stock{name:row.name})  
MERGE (c:Code{code:row.code})  
MERGE (l:Logreturn{logreturn:row.logreturn})  
MERGE (p:Price{price:row.price})  
MERGE (v:Volume{volume:row.volume})  
MERGE (t:Turnover{turnover:row.turnover})  
MERGE (s)-[r:HAS_CODE]->(c)

### some sigle node create mannuly  
MATCH (n1{ name: 'Apple' })  
create(a1:ARTICLE{content:"Apple manufactures Smartphones, tablets, PCs, software and peripherals for a worldwide customer base. Its products include Mac desktop and mobile PCs, iPhone, iPad, Apple Watch, and various consumer products, including AirPods, Beats headphones and Apple TV. Apple services include App Store, iTunes, iCloud, Apple TV+, Apple Arcade, Apple Music, Apple Pay, and more."})  
create (n1)-[r6:SAID]->(a1)

### sparqlanything method
PREFIX st: <http://ns.inria.fr/sparql-template/>  
PREFIX xyz: <http://sparql.xyz/facade-x/data/>  
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX fx: <http://sparql.xyz/facade-x/ns/>  
 
#SELECT ?stockName ?sk  
CONSTRUCT { ?stockName ?sk ?lowest_price  }  
WHERE {  

    SERVICE <x-sparql-anything:file:///Users/fusi/Desktop/ds/individual_project/demo/american_stock.json> {  
        ?s xyz:stock_code ?sk ;  
           xyz:stock_name ?stockName ;  
           xyz:lowest_price ?lowest_price ;  
           xyz:volume ?volume;  
           xyz:turnover ?turnover;  
                  #xyz:stars ?star .  
    #Filter (?star IN (xyz:LisaKudrow)).  
           
    }  

}  

### some cypher example
all relevent nodes of 'world stocks market'. 
MATCH (c:Entity{name:"World stocks market"})-[r*0..]->(result) return result  

max logreturn and relevent stock. 
MATCH (m:Logreturn)  
with max(m.logreturn) as max  
match (l{logreturn:max})  
match (l)-[:LOGRETURN]-(s:Stock)  
return s.name,max  

The article that The Fed provide is negative  
MATCH (n{name:'The Fed'})-[:SAID]->(a:ARTICLE)-[:SENTIMENT]->(e{sentiment:"NEGATIVE"})  
return count(a.content)  

# use tf-idf to calculate the similarity

In [ ]:
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA

train_set = ['Research firm Argus has reported a rise in the number of clients seeking its analysis of the UK\'s stock market in the month of November, compared with the previous 12 months, which saw a sharp fall in the value of shares across the board.']
test_set = ['Gross domestic product (GDP) fell by 0.3% in April 2022, following a fall of 0.1% in March 2022; UK GDP grew by 0.2% in the three months to April 2022.']
stopWords = stopwords.words('english')

vectorizer = CountVectorizer(stop_words = stopWords)
#print vectorizer
transformer = TfidfTransformer()
#print transformer

trainVectorizerArray = vectorizer.fit_transform(train_set).toarray()
testVectorizerArray = vectorizer.transform(test_set).toarray()
#print 'Fit Vectorizer to train set', trainVectorizerArray
#print 'Transform Vectorizer to test set', testVectorizerArray
cx = lambda a, b : round(np.inner(a, b)/(LA.norm(a)*LA.norm(b)), 3)

for vector in trainVectorizerArray:
    #print(vector)
    for testV in testVectorizerArray:
        #print(testV)
        cosine = cx(vector, testV)
        print("1")
        print(cosine)

transformer.fit(trainVectorizerArray)

transformer.transform(trainVectorizerArray).toarray()

#print(transformer.fit(testVectorizerArray))

tfidf = transformer.transform(testVectorizerArray)
#print(tfidf.todense())

## Precision-Recall Curve

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.metrics import precision_recall_curve
#from sklearn.utils.fixes import signature
plt.figure("P-R Curve")
plt.title('Precision/Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')


precision = [1,0.9677,0.8194,0.7387,0.6735,0.5323,0]
recall = [0,0.5615,0.6596,0.7550,0.8134,0.9615,1]
plt.plot(recall,precision)
plt.show()
